In [1]:
import queue
import time
import os
import sys

from DataHandler.TradeLOBHourlyDataHandler import HistoricTradeLOBHourlyDataHandler
from Strategy.LeadLagArbitrageStrategy import LeadLagArbitrageStrategy
from Portfolio.LogPlotPortfolio import LogPlotPortfolio
from Execution.execution import SimulatedExecutionHandler

event_queue = queue.Queue()
data_handler = HistoricTradeLOBHourlyDataHandler(event_queue, 
                                           symbol_list=['btc_usdt','btc_usdt'],
                                           exchange_list=['binance','bybit'], 
                                           file_dir = 'data_sample/20240101/', 
                                           # file_dir = 'data_sample/',
                                           is_csv=False)
#sys.exit()

# 组合
portfolio = LogPlotPortfolio(event_queue, data_handler)
# 回测模拟成交器；如果是实盘这里就是算法交易模块
executor = SimulatedExecutionHandler(event_queue, data_handler) 
# 策略实例。实际应用中应该有多个策略实例
strategy = LeadLagArbitrageStrategy(event_queue, data_handler, portfolio, executor,
                                    k1=0.3*1e-4,
                                    k2=0.7*1e-4,
                                    k3=1.5*1e-4,
                                    order_live_time = 25*1000,
                                    dynamic_stop_hedge = 5*1000,
                                    stop_loss_threshold = 1*1e-4,
                                    ) 

cnt = 0
while True:
    # Update the trade/LOB (specific backtest code, as opposed to live trading)
    if data_handler.continue_backtest == True:
        data_handler.update_TradeLOB()
    else:
        break
    
    # cnt+=1
    # if cnt>20000: break

    # Handle the events
    while True:
        try:
            event = event_queue.get(False)
        except queue.Empty:
            break
        else:
            if event is not None:

                if event.type == 'MARKET':
                    # print('get market event', event)
                    strategy.on_market_event(event)
                    portfolio.on_market_event(event)
                    executor.on_market_event(event)

                elif event.type == 'ORDER':
                    executor.on_order_event(event)

                elif event.type == 'FILL':
                    executor.on_fill_event(event)
                    portfolio.on_fill_event(event)
                    strategy.on_fill_event(event)
                    # if event.fill_flag=='ALL':
                    #     sys.exit()


backtest on:  ['btc_usdt_binance', 'btc_usdt_bybit']
/*----- start initialize the DataHandler -----*/
/*----- DataHandler initialization ends -----*/


In [ ]:
import pandas as pd
# 生成结果的dataframe
backtest_log = pd.DataFrame(strategy.strategy_history)
backtest_log = backtest_log.loc[backtest_log.leader_symbol=='btc_usdt_binance',].reset_index(drop=True)

backtest_log['profit'] = (backtest_log.hedge_price*(1+backtest_log.hedge_fee) - backtest_log.leader_price*(1+backtest_log.leader_fee)) * backtest_log.leader_order_qty
backtest_log['time_cost'] = backtest_log.hedge_t - backtest_log.leader_t
# print部分指标
print('样本内开仓数量:',len(backtest_log.profit))
print('Maker 成功率:', str((backtest_log.hedge_traded_is_Maker).sum()/len(backtest_log.hedge_traded_is_Maker))[:5])
print('Maker单平均成交耗时',backtest_log.loc[backtest_log.hedge_traded_is_Maker,]['time_cost'].mean(),'ms')
print('Taker单平均成交耗时',backtest_log.loc[~backtest_log.hedge_traded_is_Maker,]['time_cost'].mean(),'ms')
print('平均每单营收:',backtest_log.profit.mean())

from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import plotly.graph_objects as go
import plotly.express as px

docs_path = '_docs/'

# 创建Plotly图形
maker_time_cost = ((backtest_log.loc[backtest_log.hedge_traded_is_Maker].time_cost)/1000)
plot1 = px.histogram(maker_time_cost, x=maker_time_cost, title="plot1: Histogram of time_cost")
strategy_profit = backtest_log.profit
plot2 = px.histogram(strategy_profit, x=strategy_profit, title="plot2: Histogram of strategy_profit")

# 创建PDF
pdf_filename = docs_path+ "plots.pdf"
c = canvas.Canvas(pdf_filename, pagesize=letter)
width, height = letter

# 设置Plotly图形在PDF中的大小和位置
plot_width, plot_height = 400, 300
x_offset, y_offset = 100, height - 100

# 将Plotly图形转换为PNG图像并将其绘制到PDF上
for i, plot in enumerate([plot1, plot2]):
    plot_img_filename = docs_path+ f"plot_{i+1}.png"
    plot.write_image(plot_img_filename)
    c.drawInlineImage(plot_img_filename, x_offset, y_offset - plot_height, width=plot_width, height=plot_height)
    c.drawString(x_offset, y_offset + 10, f"Plot {i+1}")

    # 更新偏移量以便绘制下一个图形
    y_offset -= plot_height + 50

# 保存并关闭PDF文件
c.save()

样本内开仓数量: 240
Maker 成功率: 0.695
Maker单平均成交耗时 6219.269461077844 ms
Taker单平均成交耗时 19458.671232876713 ms
平均每单营收: -0.44177155050753947
